In [ ]:
import darwinex_ticks as dtw1
import pandas as pd
import numpy as np
import dateutil.relativedelta
from datetime import date
from datetime import timedelta  
import glob, os 

In [ ]:
spreads_factors=pd.read_csv("..\configs\Spreads_factors.txt", header = None, sep = '=', names = ['Pair','Spread_F'])
configs_info=pd.read_csv("..\configs\configs.txt", header = None, sep= "=", names=['Desc', 'Value'])

user= configs_info[configs_info.Desc=='user'].Value.iloc[0]
password= configs_info[configs_info.Desc=='password'].Value.iloc[0]
host_name= configs_info[configs_info.Desc=='host_name'].Value.iloc[0]
start_date= configs_info[configs_info.Desc=='start_date'].Value.iloc[0]
end_date= configs_info[configs_info.Desc=='end_date'].Value.iloc[0]
percentile= pd.to_numeric(configs_info[configs_info.Desc=='percentile'].Value.iloc[0])

In [ ]:
dwt = dtw1.DarwinexTicksConnection(dwx_ftp_user=user,
                       dwx_ftp_pass=password,
                       dwx_ftp_hostname=host_name,
                       dwx_ftp_port=21)

In [ ]:
spreads_info = pd.DataFrame(columns=['Pair', 'Spread'])

total_pair = pd.DataFrame(dwt._ftpObj.nlst()[2:])[0]

In [ ]:
path = r'C:\Users\Isabel Cristina\Desktop\Algo_trading_py\data\*.csv'
csv_files = reports = glob.glob(path)
csv_files

In [ ]:
start_date = "2019-01-01 00"
end_date = "2019-01-01 23"
print("Total files: {}".format(len(csv_files)))
i = 1
file = csv_files[1]
for file in csv_files:
    pair = file.split("\\")[-1][0:6]
    print("\n--- CREATING DATA FOR FILE {}: {} ---\n".format(i, pair))

    # Reading data from darwinex
    print("Dowloading from {} to {}".format(start_date, end_date))
    data = dwt.ticks_from_darwinex(pair, start=start_date, end=end_date)
    data['Price'] = (data.Ask + data.Bid) / 2
    data_ohlc = data['Price'].resample('1Min').ohlc()

    # Date for filtering
    start_date_filter = pd.to_datetime('2006-07-01').tz_localize('UTC')
    min_date = min(data_ohlc.index)
    # min_hour = data_ohlc.Hour[0]
    max_date = max(data_ohlc.index)

    # Reading data from mt4
    print("Reading csv")
    data_mt = pd.read_csv(file, header=None)

    # Procesing data
    data_mt.columns = ["Date", "Hora", "open", "high", "low", "close", "Vol"]
    data_mt['Time'] = pd.to_datetime(data_mt['Date'] + ' ' + data_mt['Hora'])
    data_mt = data_mt[['Time', 'open', 'high', 'low', 'close']].set_index('Time')

    # Setting time zone
    data_mt.index = data_mt.index.tz_localize('UTC')
    data_mt.index = data_mt.index - pd.Timedelta(hours=2)
    data_mt = data_mt.loc[start_date_filter:min_date]

    # Merging data
    data_total = data_mt.append(data_ohlc).reset_index()
    data_total['Date'] = data_total.Time.dt.strftime('%Y.%m.%d')
    data_total['Hour'] = data_total.Time.dt.strftime('%H:%M')
    data_total = data_total[['Date', 'Hour', 'open', 'high', 'low', 'close']]

    # Saving csv data
    print("Saving update data...")
    data_total.to_csv("..\data\data_updated\{}_from_{}_to_{}.csv".format(pair, str(start_date).split(sep=" ")[0],
                                                    str(max_date).split(sep=" ")[0]), header=False,
                      index=False)

    i += 1
    del data, data_ohlc, data_total, data_mt
print ("\n--- DONE ---")

In [ ]:
dwt.list_of_files('EURGBP').head(10)

In [ ]:
dwt.ticks_from_darwinex('EURGBP', start='2017-10-01 00', end='2017-10-02 23', verbose=True)

In [ ]:
data_1 = data.copy()
# data_1 = data_1.reset_index()
# data_1['Time'] = pd.to_datetime(data_1['Time'])
data_1.index = pd.to_datetime(data_1.index)
data_1.info()

In [ ]:
data_1['Price'] = (data_1.Ask+data_1.Bid)/2
data_ohlc = data_1['Price'].resample('1Min').ohlc()
# data_ohlc = data_ohlc.reset_index()
data_ohlc

In [ ]:
# data_ohlc['Date'] = data_ohlc['Time'].dt.strftime('%Y/%m/%d')
# data_ohlc['Hour'] = data_ohlc['Time'].dt.strftime('%H:%M')

In [ ]:
# data_ohlc = data_ohlc[['Date', 'Hour', 'open', 'high', 'low', 'close']]
data_ohlc.info()

In [ ]:
start_date = pd.to_datetime('2006-07-01').tz_localize('UTC')
min_date = min(data_ohlc.index)
# min_hour = data_ohlc.Hour[0]
max_date = max(data_ohlc.index)

In [ ]:
data_mt = pd.read_csv("EURUSD1.csv", header = None)
data_aux = data_mt.copy()

In [ ]:
data_mt = data_aux.copy()
data_mt.columns = ["Date", "Hora", "open", "high", "low", "close", "Vol"]
data_mt['Time'] = pd.to_datetime(data_mt['Date'] + ' ' + data_mt['Hora'])
# data_mt['Date'] = pd.to_datetime(data_mt.Date)
# data_mt['Hora'] = pd.to_timestamp(data_mt.Hora, format= '%H:%M' )#.dt.time

In [ ]:
# data_mt['Hora'] = data_mt.Hora - pd.Timedelta(hours=2)
# data_mt['Hora'].to_timestamp
# data_mt

In [ ]:
# data_mt = data_mt[data_mt.Date >= start_date].reset_index()
# data_mt[(data_mt.Date < min_date) & (data_mt.Hora < min_hour)]

In [ ]:
data_mt = data_mt[['Time', 'open', 'high', 'low', 'close']].set_index('Time')
data_mt.index = data_mt.index.tz_localize('UTC')
data_mt

In [ ]:
data_mt.index = data_mt.index - pd.Timedelta(hours=2)
data_mt

In [ ]:
data_mt = data_mt.loc[start_date:min_date]
data_mt

In [ ]:
data_total = data_mt.append(data_ohlc).reset_index()
data_total['Date'] = data_total.Time.dt.strftime('%Y.%m.%d')
data_total['Hour'] = data_total.Time.dt.strftime('%H:%M')
data_total

In [ ]:
data_total = data_total[['Date', 'Hour', 'open', 'high', 'low', 'close']]

In [ ]:
# data_total['Time'] = data_total['Time'].astype(str)

In [ ]:
# data_total['Time'] = data_total['Time'].split(sep = "+")

In [ ]:
# pd.to_datetime(data_total.index, format='%Y.%m.%d %H:%M')
# data_total.index = data_total.index.strftime('%Y.%m.%d %H:%M').reet_index()
# data_total['Time'].str.split("+")[0]

In [ ]:
data_total

In [ ]:
data_total.to_csv("eurusd_{}_{}.csv".format(str(start_date).split(sep = " ")[0], 
                                            str(max_date).split(sep = " ")[0]), header = False, 
                  index  = False)

In [ ]:
data_total= data_total.index.str.split(expand=True) 


In [ ]:
pd.to_datetime('2007-01-01').tz_localize('Etc/GMT+2')

In [ ]:
for pair in total_pair:
    spread_factor=spreads_factors[spreads_factors.Pair==pair].Spread_F
    spread_info = pd.DataFrame()
    
    try:
        data = dwt.ticks_from_darwinex(pair, start=start_date, end=end_date)
    except:
        pass
    
    data_spread= dtw1.spread(data)
    
    try:
        spread_pair = [round((data_spread*spread_factor.iloc[0]).quantile(percentile),5)]
    except:
        spread_pair = ['not in spread factor´s file, data spread : '+ str(round(np.max(data_spread),5))]
        
    print(pair," = ", spread_pair)
    spread_info = pd.DataFrame({'Pair' : [pair],
                           'Spread' : spread_pair})
    spreads_info = spreads_info.append(spread_info, ignore_index=True)
    
 

str_pair = ''
for i in range(0, (len(spreads_info))): 
        str_pair = str_pair + spreads_info['Pair'][i] + '='+ str(spreads_info['Spread'][i]) + '\n'

file = open('pairs.txt','w') 
 
file.write(str_pair) 
 
file.close() 

In [ ]:
np.max(data_spread[1:4])
data_spread